In [8]:
import pandas as pd 
from pathlib import Path
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import sqlite3

In [9]:
# ETL

In [10]:
# Read in the CSV file as a Pandas Dataframe
df = pd.read_csv(Path("resources/baseballData.csv"))
#ridding csv of partial seasons
dfFullYear = df[~df['year'].isin([2020, 2024])]
#dropping unesscessary columns
#'babip','bacon',
dfFullYear.drop(columns=["player_id",'player_age', "last_name, first_name", 'year', 'ab', 'pa', 'hit', 'single', 'double', 'triple', 'out_zone_swing_miss', 'babip','bacon',
       'out_zone_swing', 'out_zone_percent', 'out_zone', 'in_zone_swing_miss', 'single', 'double', 'triple', 'home_run', 'strikeout', 'walk', 'bb_percent', 'on_base_percent', 'on_base_plus_slg', 'isolated_power','woba','wobacon',
       'in_zone_swing','batted_ball', 'groundballs', 'flyballs', 'linedrives', 'popups',
       'home_run', 'strikeout', 'walk'], inplace=True)
dfFullYear.columns
dfFullYear.head(10)

/var/folders/xn/3pkq9rt94zb_qff3dyqz_mmc0000gn/T/ipykernel_67495/1161540600.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfFullYear.drop(columns=["player_id",'player_age', "last_name, first_name", 'year', 'ab', 'pa', 'hit', 'single', 'double', 'triple', 'out_zone_swing_miss', 'babip','bacon',


,k_percent,batting_avg,slg_percent,exit_velocity_avg,launch_angle_avg,sweet_spot_percent,barrel,barrel_batted_rate,solidcontact_percent,flareburner_percent,...,whiff_percent,swing_percent,pull_percent,straightaway_percent,opposite_percent,f_strike_percent,groundballs_percent,flyballs_percent,linedrives_percent,popups_percent
0,18.5,0.240,0.409,88.5,10.8,28.5,21,5.0,5.2,21.6,...,23.1,53.4,34.7,40.6,24.5,64.0,49.4,22.6,20.9,7.1
1,15.5,0.273,0.553,93.0,15.7,34.8,58,13.1,8.4,24.9,...,23.2,44.7,41.9,36.4,21.7,52.1,37.6,25.6,28.7,8.1
2,23.4,0.250,0.486,91.3,12.2,31.4,42,10.9,8.1,20.3,...,32.0,43.9,38.4,43.9,16.9,60.3,43.6,24.9,24.9,6.5
3,13.2,0.246,0.423,87.4,15.8,33.5,23,5.6,6.6,23.3,...,17.9,52.9,36.9,38.3,24.8,65.5,37.6,24.5,29.1,8.7
4,10.5,0.287,0.453,89.5,12.6,35.7,28,5.5,7.8,26.3,...,16.8,48.1,35.3,41.4,23.3,61.9,42.4,18.4,31.6,7.6
5,16.0,0.276,0.471,90.6,15.6,34.1,23,5.8,7.5,27.3,...,18.1,45.4,40.6,38.3,19.3,57.3,36.8,26.6,28.1,8.5
6,15.3,0.224,0.358,85.7,16.2,32.1,11,2.5,4.2,23.8,...,18.1,45.0,46.0,32.6,21.2,56.5,41.3,24.7,24.2,9.7
7,10.9,0.244,0.480,90.7,14.2,29.5,41,7.7,5.3,22.7,...,16.3,45.4,43.3,37.0,19.5,56.0,42.8,25.5,23.8,7.9
8,26.7,0.247,0.453,86.3,12.1,33.9,31,8.5,5.5,21.9,...,31.2,60.2,36.6,35.8,27.6,70.4,43.7,22.7,28.7,4.9
9,16.8,0.265,0.380,89.5,4.4,33.3,18,3.7,8.0,28.6,...,16.8,40.7,28.2,39.7,32.1,57.2,57.6,13.4,27.2,1.9


In [11]:
dfFullYear.to_csv('Resources/Baseball_stats_cleaned.csv')

In [12]:
# SQL DB

In [14]:
# Step1. Load data file
df_cleaned = pd.read_csv('Resources/Baseball_stats_cleaned.csv')

In [15]:
# Step2. Create/connect to SQLite DB.
connection = sqlite3.connect('stats.db')

In [16]:
# Step3. Load dataset to SQLite DB
df_cleaned.to_sql('baseball_stats', connection, if_exists = 'replace')

1104

In [17]:
#Scale the Data
scaler = StandardScaler()
scaler.fit(dfFullYear)

StandardScaler()

In [18]:
#Assign Variables to be Trained
X = df_cleaned.drop('batting_avg', axis=1)
y = df_cleaned["batting_avg"]

In [19]:
#Split the data into training & testing
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state = 50)


In [20]:
#Create a Rabndom Forest Regressor Model
rf_model = RandomForestRegressor(n_estimators=100, random_state=50)

In [21]:
#Train the Model
rf_model.fit(X_train, y_train)

RandomForestRegressor(random_state=50)

In [22]:
#Make Predictions
predictions = rf_model.predict(X_test)
print(predictions)


[0.28759 0.2912  0.23088 0.29157 0.29308 0.25297 0.26886 0.2749  0.22979
 0.27148 0.29614 0.27241 0.28167 0.27078 0.25263 0.24711 0.27955 0.2759
 0.28709 0.28044 0.26399 0.25781 0.27644 0.23877 0.25475 0.30886 0.24678
 0.25417 0.25728 0.26017 0.27606 0.2731  0.24419 0.27987 0.26511 0.25571
 0.25256 0.23232 0.27932 0.25002 0.27019 0.24671 0.26865 0.31424 0.27055
 0.26752 0.2702  0.29337 0.27502 0.25246 0.25172 0.27812 0.24695 0.2511
 0.27966 0.27611 0.29246 0.23978 0.25517 0.26801 0.28719 0.29204 0.27796
 0.27029 0.2617  0.2896  0.24494 0.24617 0.28425 0.24773 0.28138 0.22931
 0.25068 0.25957 0.2588  0.2368  0.27074 0.24653 0.24066 0.28829 0.26087
 0.25098 0.25991 0.26385 0.27829 0.27122 0.28339 0.25701 0.27169 0.27367
 0.28311 0.26208 0.25856 0.23463 0.23976 0.30164 0.25763 0.28789 0.31596
 0.25585 0.23737 0.26237 0.30367 0.30355 0.25762 0.25478 0.24246 0.26258
 0.26794 0.28923 0.28465 0.25954 0.31238 0.27424 0.25818 0.30389 0.25277
 0.22877 0.28342 0.27685 0.2851  0.26548 0.29384 0.27

In [23]:
mean = mean_absolute_error(y_test, predictions)
print("Mean", mean)

Mean 0.011456923076923088


In [24]:
# Close SQL connection
connection.close()